In [1]:
from vespa.application import Vespa

app = Vespa(url = "https://api.cord19.vespa.ai")

In [2]:
body = {
  'yql': 'select title, abstract from sources * where userQuery();',
  'hits': 5,
  'query': 'Is remdesivir an effective treatment for COVID-19?',
  'type': 'any',
  'ranking': 'bm25'
}

In [3]:
results = app.query(body=body)

In [4]:
results.number_documents_retrieved

268858

In [16]:
for result in results.hits:
    print(result['fields']['title'])

Probable Molecular Mechanism of <hi>Remdesivir</hi> <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>: Need to Know More
Probable Molecular Mechanism of <hi>Remdesivir</hi> <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>: Need to Know More
Probable Molecular Mechanism of <hi>Remdesivir</hi> <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>: Need to Know More
A Review on <hi>Remdesivir</hi>: A Possible Promising Agent <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>
A Review on <hi>Remdesivir</hi>: A Possible Promising Agent <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>
<hi>Remdesivir</hi> <hi>for</hi> severe <hi>covid</hi>-<hi>19</hi>: a clinical practice guideline.
<hi>Remdesivir</hi> <hi>for</hi> severe <hi>covid</hi>-<hi>19</hi>: a clinical practice guideline
<hi>Remdesivir</hi> <hi>for</hi> the <hi>Treatment</hi> of <hi>COVID</hi>-<hi>19</hi>: A Systematic Review of the Literature
<hi>Remdesivir<

In [5]:
from vespa.query import Query, OR, RankProfile

results = app.query(
    query="Is remdesivir an effective treatment for COVID-19?",
    query_model = Query(
        match_phase=OR(),
        rank_profile=RankProfile(name="bm25")
    )
)

In [6]:
results.number_documents_retrieved

268858

In [7]:
from vespa.query import Query, ANN, WeakAnd, Union, RankProfile
from random import random

match_phase = Union(
    WeakAnd(hits = 10),
    ANN(
        doc_vector="title_embedding",
        query_vector="title_vector",
        embedding_model=lambda x: [random() for x in range(768)],
        hits = 10,
        label="title"
    )
)
rank_profile = RankProfile(name="bm25", list_features=True)
query_model = Query(match_phase=match_phase, rank_profile=rank_profile)

In [8]:
results = app.query(query="Is remdesivir an effective treatment for COVID-19?",
          query_model=query_model)

In [9]:
results.number_documents_retrieved

1121

In [10]:
top_ids = [hit["fields"]["id"] for hit in results.hits[0:3]]
top_ids

[269386, 144385, 144384]

In [11]:
results_with_recall = app.query(query="Is remdesivir an effective treatment for COVID-19?",
                    query_model=query_model,
                    recall = ("id", top_ids[1:3]))

In [12]:
id_recalled = [hit["fields"]["id"] for hit in results_with_recall.hits]
id_recalled

[144385, 144384]